In [1]:
import pandas as pd
import numpy as np
import time
from tqdm import tqdm
import datetime as datetime

In [2]:
weathe=pd.read_excel('./测试集B/2017年7月城市天气详单.xlsx')
train=pd.read_csv('./测试集B/201707测试数据.csv',encoding='gbk')
airport=pd.read_excel('./测试集B/机场城市对应表.xlsx')
news=pd.read_excel('./测试集B/2017年7月特情数据详单.xlsx')

# 对测试集B的测试数据进行整理，方法与对测试集A的方法相同

In [3]:
def train_data_get(df):
    time_list=['计划起飞时间','计划到达时间']
    for j in time_list:
        tmp=df[j].fillna(0).astype('int64')
        stmp=[]
        for i in tqdm(tmp):
            timelocal=time.localtime(i)
            stmp.append(time.strftime('%Y-%m-%d %H:%M:%S', timelocal))
        df[j+'stand']=stmp
    df['company']=df['航班编号'].apply(lambda x: x[:2])
    df['from']=df['航班编号'].apply(lambda x: x[2:3])
    df['to']=df['航班编号'].apply(lambda x: x[3:4])
    df['times']=df['航班编号'].apply(lambda x: x[4:5])
    df['go_to']=df['航班编号'].apply(lambda x: x[5:])
    df['year']=df['计划起飞时间stand'].apply(lambda x: x[:4])
    df['month']=df['计划起飞时间stand'].apply(lambda x: x[5:7])
    df['day']=df['计划起飞时间stand'].apply(lambda x: x[8:10])
    df['take_hour']=df['计划起飞时间stand'].apply(lambda x: x[11:13])
    df['take_minute']=df['计划起飞时间stand'].apply(lambda x: x[14:16])
    df['off_hour']=df['计划到达时间stand'].apply(lambda x: x[11:13])
    df['off_minute']=df['计划到达时间stand'].apply(lambda x: x[14:16])    
    df['出发日期']=pd.to_datetime(df['计划起飞时间stand'].apply(lambda x: x[:10]))
    df['到达日期']=pd.to_datetime(df['计划到达时间stand'].apply(lambda x: x[:10]))
    train_data=df.drop(['实际起飞时间','实际到达时间','取消标识'],axis=1)
    return train_data

In [4]:
def weather_data_get(airport_info,weathe):
    airport_info.rename(columns={'城市名称':'城市'},inplace=True)
    weathe.rename(columns={'机场':'城市'},inplace=True)
    weather_data=weathe.drop_duplicates(['城市','日期'])
    new_weather_data=airport_info.merge(weather_data,on='城市',how='left')
    return new_weather_data
def star_weather_data_get(df):
    star_weather_data=pd.DataFrame()
    star_weather_data['出发机场']=df['机场编码']
    star_weather_data['出天气']=df['天气']
    star_weather_data['出min']=df['最低气温']
    star_weather_data['出max']=df['最高气温']
    star_weather_data['出发日期']=pd.to_datetime(new_weather_data['日期'])
    return star_weather_data
def arrival_weather_data_get(df):
    arrive_weather_data=pd.DataFrame()
    arrive_weather_data['到达机场']=df['机场编码']
    arrive_weather_data['到天气']=df['天气']
    arrive_weather_data['到min']=df['最低气温']
    arrive_weather_data['到max']=df['最高气温']
    arrive_weather_data['到达日期']=pd.to_datetime(df['日期'])
    return arrive_weather_data

In [5]:
def news_data_get(df):
    list_1=df['机场'].astype('str')
    list_2=[]
    for i in tqdm(list_1):
        u=i.upper()
        list_2.append(u)
    df['机场编码']=list_2
    df['日期']=pd.to_datetime(df['开始时间'].astype('str').apply(lambda x: x[:10]))
    df['收集时间stand']=pd.to_datetime(df['采集时间'])
    df['star']=pd.to_datetime(df['开始时间'])
    df['end']=pd.to_datetime(df['结束时间'])
    df['差值']=(df['end']-df['star'])
    df=df[(df['差值']>='00:00:00')&(df['差值']<='23:59:59')]
    df['hour']=df['差值'].astype('str').apply(lambda x: x[7:9]).astype(int)
    df['minutes']=df['差值'].astype('str').apply(lambda x: x[10:12]).astype(int)
    df['set']=df['hour']*60+df['minutes']  
    return df
def star_news_get(df):
    star_news_data=pd.DataFrame()
    star_news_data['出发机场']=df['机场编码']
    star_news_data['出set']=df['set']
    star_news_data['出收集时间']=df['收集时间stand']
    star_news_data['出开始']=df['star']
    star_news_data['出结束']=df['end']
    star_news_data['出发日期']=df['日期']
    return star_news_data
def arrival_news_get(df):
    arrive_news_data=pd.DataFrame()
    arrive_news_data['到达机场']=df['机场编码']
    arrive_news_data['到set']=df['set']
    arrive_news_data['到收集时间']=df['收集时间stand']
    arrive_news_data['到开始']=df['star']
    arrive_news_data['到结束']=df['end']
    arrive_news_data['到达日期']=df['日期']
    return arrive_news_data

In [6]:
def data_organization(train_data,star_new,arrive_new,star_weather,arrive_weather):
    print(train_data.shape)
    train_data_1=pd.merge(train_data,star_weather,on=['出发机场','出发日期'],how='left')
    print(train_data_1.shape)
    train_data_2=pd.merge(train_data_1,arrive_weather,on=['到达机场','到达日期'],how='left')
    print(train_data_2.shape)
    train_data_3=pd.merge(train_data_2,star_new,on=['出发机场','出发日期'],how='left')
    print(train_data_3.shape)
    train_data_4=pd.merge(train_data_3,arrive_new,on=['到达机场','到达日期'],how='left')
    print(train_data_4.shape)
    return train_data_4

In [7]:
train_data=train_data_get(train)

100%|██████████████████████████████| 183931/183931 [00:01<00:00, 116184.76it/s]


In [8]:
new_weather_data=weather_data_get(airport,weathe)

In [9]:
star_weather_data=star_weather_data_get(new_weather_data)

In [10]:
arrival_weather_data=arrival_weather_data_get(new_weather_data)

In [11]:
news_data=news_data_get(news)

100%|████████████████████████████████████████████████| 108/108 [00:00<?, ?it/s]


In [12]:
star_news_data=star_news_get(news_data)

In [13]:
arrival_news_data=arrival_news_get(news_data)

In [14]:
train_data_1=train_data[train_data['验证标识（1：需要选手预测；0：提前两小时航班信息参考数据）']==1]

In [15]:
test_x=data_organization(train_data_1,star_news_data,arrival_news_data,star_weather_data,arrival_weather_data)

(21368, 23)
(21368, 26)
(21368, 29)
(21793, 33)
(22271, 37)


In [16]:
test_x['出_rule']=(pd.to_datetime(test_x['计划起飞时间stand'])-pd.to_datetime(test_x['出收集时间'])).fillna(0)
test_x['到_rule']=pd.to_datetime(test_x['计划起飞时间stand'])-pd.to_datetime(test_x['到收集时间']).fillna(0)
test_x['出_上界']=(pd.to_datetime(test_x['出开始'])-pd.to_datetime(test_x['计划起飞时间stand'])).fillna(0)
test_x['出_下界']=(pd.to_datetime(test_x['出结束'])-pd.to_datetime(test_x['计划起飞时间stand'])).fillna(0)
test_x['到_上界']=pd.to_datetime(test_x['出开始'])-pd.to_datetime(test_x['计划到达时间stand']).fillna(0)   #testB问题所在
test_x['到_下界']=pd.to_datetime(test_x['出结束'])-pd.to_datetime(test_x['计划到达时间stand']).fillna(0)  #testB问题所在

In [17]:
list_1=[]
for i in range(len(test_x)):
    if (test_x['出_rule'].values[i]>=np.timedelta64(7200000000000,'ns'))&(test_x['出_上界'].values[i]<=np.timedelta64(0,'ns'))&(test_x['出_下界'].values[i]>=np.timedelta64(0,'ns')):
        list_1.append(1)
    else:
        list_1.append(0)
test_x['出_inflect']=list_1

In [18]:
list_2=[]
for i in range(len(test_x)):
    if (test_x['到_rule'].values[i]>=np.timedelta64(7200000000000,'ns'))&(test_x['到_上界'].values[i]<=np.timedelta64(0,'ns'))&(test_x['到_下界'].values[i]>=np.timedelta64(0,'ns')):
        list_2.append(1)
    else:
        list_2.append(0)
test_x['到_inflect']=list_2

D:\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: FutureWarning: In the future, 'NAT <= x' and 'x <= NAT' will always be False.
  app.launch_new_instance()


In [19]:
train_new_x=test_x[['出发机场', '到达机场', '航班编号', '计划起飞时间', '计划到达时间', '飞机编号',
       '计划起飞时间stand', '计划到达时间stand', 'company', 'from', 'to', 'times',
       'go_to', 'year', 'month', 'day', 'take_hour', 'take_minute',
       'off_hour', 'off_minute', '出发日期', '到达日期', '出天气', '出min', '出max',
       '到天气', '到min', '到max', '出set',  '到set',
       '出_inflect', '到_inflect']]

In [20]:
train_new_x.to_csv('testB_ready.csv',sep=',',header=True,index=False)